In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import mnist_loader
import random

In [3]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [4]:
class Network:
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.rand(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data is not None:
            n_test = len(test_data)
        n = len(training_data)
        for j in np.arange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k + mini_batch_size] for k in np.arange(0, n, mini_batch_size)
            ]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data is not None:
                print("Epoch {0}, {1} / {2}"
                .format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = np.array([np.zeros(b.shape) for b in self.biases])
        nabla_w = np.array([np.zeros(w.shape) for w in self.weights])
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            print(delta_nabla_b, delta_nabla_w)
            nabla_w = nabla_w + delta_nabla_w
            nabla_b = nabla_b + delta_nabla_b
            
        self.weights = self.weights + eta * nabla_w / len(mini_batch)
        self.biases = self.biases + eta * nabla_b / len(mini_batch)

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in np.arange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [5]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [6]:
net = Network([784, 30, 10])

In [7]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0, 974 / 10000
Epoch 1, 974 / 10000
Epoch 2, 974 / 10000
Epoch 3, 974 / 10000
Epoch 4, 974 / 10000
Epoch 5, 974 / 10000
Epoch 6, 974 / 10000
Epoch 7, 974 / 10000
Epoch 8, 974 / 10000
Epoch 9, 974 / 10000
Epoch 10, 974 / 10000
Epoch 11, 974 / 10000
Epoch 12, 974 / 10000
Epoch 13, 974 / 10000
Epoch 14, 974 / 10000
Epoch 15, 974 / 10000
Epoch 16, 974 / 10000
Epoch 17, 974 / 10000
Epoch 18, 974 / 10000
Epoch 19, 974 / 10000
Epoch 20, 974 / 10000
Epoch 21, 974 / 10000
Epoch 22, 974 / 10000
Epoch 23, 974 / 10000
Epoch 24, 974 / 10000
Epoch 25, 974 / 10000
Epoch 26, 974 / 10000
Epoch 27, 974 / 10000
Epoch 28, 974 / 10000
Epoch 29, 974 / 10000
